In [1]:
from glob import glob
import json
import pickle

import cv2
from imageio import imread
# import keras
import tensorflow as tf
# from tensorflow.keras import callbacks, optimizers, utils
# from tensorflow.keras.applications import Xception
# from tensorflow.keras.applications.imagenet_utils import preprocess_input
# from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

import bottleneck
from keras import backend as K
import matplotlib.patches as patches
from scipy.special import softmax
from tensorflow import keras
# from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Add, Concatenate, Conv2D, DepthwiseConv2D, Input, Reshape

# %matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))


2022-09-29 01:30:30.213011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 01:30:30.334068: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 01:30:30.919420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/z/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-09-29 01:30:30.919518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dle

# 1. INPUT

### 1-1. SETUP

In [2]:
NUM_CLASSES = 2
INPUT_SHAPE = (80, 80, 1)
BATCH_SIZE = 32

In [3]:
# priors = pickle.load(open('priorFiles/prior_boxes_ssd300VGG16.pkl', 'rb'))
# bbox_util = BBoxUtility(NUM_CLASSES, priors)
# priors

In [4]:
# path = '~/MVPC10/DATA/mvpc10/'
# csv = f'IN/label_467.csv'
csv = f'IN/labeled.csv'

df = pd.read_csv(csv, encoding='utf-8')
df.head

<bound method NDFrame.head of                data                                              label
0     1650315736806                                                 []
1     1650315796123                                                 []
2     1650315856514                                                 []
3     1650315917006                                                 []
4     1650315976418                                                 []
...             ...                                                ...
7846  1652071876733                 [[28, 11, 39, 32], [30, 0, 35, 6]]
7847  1652071936097                  [[30, 0, 35, 6], [39, 4, 47, 18]]
7848  1652071996499                 [[36, 12, 46, 34], [31, 1, 34, 5]]
7849  1652072056846  [[39, 35, 56, 59], [18, 18, 29, 36], [29, 5, 3...
7850  1652072116028  [[32, 62, 54, 80], [47, 36, 63, 60], [15, 18, ...

[7851 rows x 2 columns]>

In [5]:
df1 = df.iloc[:,0]
df1.head

<bound method NDFrame.head of 0       1650315736806
1       1650315796123
2       1650315856514
3       1650315917006
4       1650315976418
            ...      
7846    1652071876733
7847    1652071936097
7848    1652071996499
7849    1652072056846
7850    1652072116028
Name: data, Length: 7851, dtype: int64>

In [6]:
## LABEL COLUMN: STR TO LIST

# df = json.loads(df.iloc[0,1])
# json_loc = json.dumps(bbox_list)

df2 = df.iloc[:,1].apply(json.loads)
df2.head


<bound method NDFrame.head of 0                                                      []
1                                                      []
2                                                      []
3                                                      []
4                                                      []
                              ...                        
7846                   [[28, 11, 39, 32], [30, 0, 35, 6]]
7847                    [[30, 0, 35, 6], [39, 4, 47, 18]]
7848                   [[36, 12, 46, 34], [31, 1, 34, 5]]
7849    [[39, 35, 56, 59], [18, 18, 29, 36], [29, 5, 3...
7850    [[32, 62, 54, 80], [47, 36, 63, 60], [15, 18, ...
Name:  label, Length: 7851, dtype: object>

In [7]:
df2

0                                                      []
1                                                      []
2                                                      []
3                                                      []
4                                                      []
                              ...                        
7846                   [[28, 11, 39, 32], [30, 0, 35, 6]]
7847                    [[30, 0, 35, 6], [39, 4, 47, 18]]
7848                   [[36, 12, 46, 34], [31, 1, 34, 5]]
7849    [[39, 35, 56, 59], [18, 18, 29, 36], [29, 5, 3...
7850    [[32, 62, 54, 80], [47, 36, 63, 60], [15, 18, ...
Name:  label, Length: 7851, dtype: object

In [8]:
## ADD DF and DF1

df = pd.concat([df1, df2], axis=1, join='inner')
df.head

<bound method NDFrame.head of                data                                              label
0     1650315736806                                                 []
1     1650315796123                                                 []
2     1650315856514                                                 []
3     1650315917006                                                 []
4     1650315976418                                                 []
...             ...                                                ...
7846  1652071876733                 [[28, 11, 39, 32], [30, 0, 35, 6]]
7847  1652071936097                  [[30, 0, 35, 6], [39, 4, 47, 18]]
7848  1652071996499                 [[36, 12, 46, 34], [31, 1, 34, 5]]
7849  1652072056846  [[39, 35, 56, 59], [18, 18, 29, 36], [29, 5, 3...
7850  1652072116028  [[32, 62, 54, 80], [47, 36, 63, 60], [15, 18, ...

[7851 rows x 2 columns]>

In [9]:

a = df.iloc[69,1]
print(type(a))
print(a[0])


<class 'list'>
[45, 33, 58, 54]


### 1-2. PRE-PROCESS

In [10]:
data_base_path = f"/media/z/0/MVPC10/DATA/v1.1/RAW/03"

path = []
data = []
label = []
# data = np.empty((80,80))
for i in range(len(df.index)):
    label_list = df.iloc[i,1]
    if len(label_list) > 0:
    # if len(label_list) > 1:
        if label_list[0] == -1:  continue
        data_path = glob(f"{data_base_path}/**/{df.iloc[i, 0]}.png")
        try:
            data_arr = cv2.imread(data_path[0], 0)
        except IndexError as IE:
            print(f"ERROR FILE: {df.iloc[i, 0]}")
        # data_arr = Image.open(data_path[0]).convert('L')
        # data_arr = np.asarray(data_arr)
        for l in label_list:
            path.append(df.iloc[i, 0])
            data.append(data_arr)
            label.append(l)

path = np.array(path)
data = np.array(data)#, dtype=float)
label = np.array(label, dtype=object)

print(path.shape)
print(data.shape)
print(label.shape)

(13787,)
(13787, 80, 80)
(13787, 4)


In [11]:
## SAVE TO NEW CSV

# new_list = []
# for a, b in zip(path, label):
#     new_list.append([a,b])
# new_list = np.array(new_list, dtype=object)
# new_data = {'path': path, 'label': label}

new_df = pd.DataFrame(list(zip(path, label)), columns=['path', 'label'])
new_df.to_csv('new_label.csv', index=False)

In [12]:
# dataset = 'dataset.pkl'
# df.to_pickle(dataset)
# df1 = pd.read_pickle(dataset)
# print(df1.head())

### 1-2. NORMALIZE

In [13]:
MIN, X_MAX, Y_MAX = 0, 255, 79
data = (data / X_MAX).astype('float16')
label = (label / Y_MAX).astype('float16')

print(f"data:{chr(10)}{data}")
print(f"label:{chr(10)}{label}")
print(f"label.shape:{chr(10)}{label.shape}")

data:
[[[0.902  0.898  0.8706 ... 0.851  0.8667 0.8823]
  [0.9175 0.894  0.863  ... 0.851  0.8433 0.894 ]
  [0.937  0.906  0.8667 ... 0.8433 0.859  0.894 ]
  ...
  [0.9727 0.9214 0.9136 ... 0.898  0.9136 0.949 ]
  [0.9805 0.929  0.9097 ... 0.902  0.9175 0.961 ]
  [0.953  0.9253 0.902  ... 0.89   0.906  0.937 ]]

 [[0.8237 0.8037 0.82   ... 0.8784 0.906  0.9253]
  [0.8237 0.8076 0.8076 ... 0.886  0.89   0.929 ]
  [0.859  0.8076 0.8354 ... 0.8706 0.898  0.9453]
  ...
  [0.8745 0.8237 0.8706 ... 0.929  0.957  0.9883]
  [0.886  0.8237 0.894  ... 0.9453 0.949  0.9883]
  [0.8706 0.8237 0.855  ... 0.933  0.937  0.9844]]

 [[0.8115 0.8237 0.8433 ... 0.89   0.9453 0.929 ]
  [0.8037 0.8354 0.8433 ... 0.886  0.929  0.9453]
  [0.8354 0.8276 0.855  ... 0.89   0.949  0.953 ]
  ...
  [0.851  0.855  0.8823 ... 0.953  0.9883 0.9883]
  [0.8745 0.8667 0.9097 ... 0.9688 0.9883 0.9883]
  [0.8433 0.863  0.886  ... 0.953  0.9883 0.9883]]

 ...

 [[0.612  0.6353 0.6    ... 0.608  0.678  0.71  ]
  [0.6157 0.62

### ADD CLASS to LABEL

In [14]:
lbl_cls = np.ones((label.shape[0], 1))
cls_label = np.concatenate((lbl_cls, label), axis=1)
cls_label.shape

(13787, 5)

### 1-3. RESIZE

In [15]:
# data1 = np.expand_dims(data, axis=-1)
data1 = data.reshape(data.shape[0], data.shape[1], data.shape[2], 1)
# data1 = data.reshape(-1, 80, 80, 1)
print(data1.shape)
print(data1[0])

(13787, 80, 80, 1)
[[[0.902 ]
  [0.898 ]
  [0.8706]
  ...
  [0.851 ]
  [0.8667]
  [0.8823]]

 [[0.9175]
  [0.894 ]
  [0.863 ]
  ...
  [0.851 ]
  [0.8433]
  [0.894 ]]

 [[0.937 ]
  [0.906 ]
  [0.8667]
  ...
  [0.8433]
  [0.859 ]
  [0.894 ]]

 ...

 [[0.9727]
  [0.9214]
  [0.9136]
  ...
  [0.898 ]
  [0.9136]
  [0.949 ]]

 [[0.9805]
  [0.929 ]
  [0.9097]
  ...
  [0.902 ]
  [0.9175]
  [0.961 ]]

 [[0.953 ]
  [0.9253]
  [0.902 ]
  ...
  [0.89  ]
  [0.906 ]
  [0.937 ]]]


In [16]:
label1 = cls_label.reshape(cls_label.shape[0], cls_label.shape[1], 1)
# label1 = np.expand_dims(label, axis=-1)

print(label1.shape)
print(label1[0])

(13787, 5, 1)
[[1.        ]
 [0.25317383]
 [0.17724609]
 [0.4050293 ]
 [0.31640625]]


In [17]:
# label1.shape

### 1-4. SPLIT

In [18]:
VAL_SIZE = 0.15
TEST_SIZE = 0.08

VAL_SIZE = round(data1.shape[0]*(VAL_SIZE))
TEST_SIZE = round((data1.shape[0]-VAL_SIZE)*(TEST_SIZE))

print(VAL_SIZE)
print(TEST_SIZE)

2068
938


In [19]:
SPLIT_1 = data1.shape[0]-(VAL_SIZE+TEST_SIZE)
SPLIT_2 = SPLIT_1+VAL_SIZE
SPLIT_3 = SPLIT_1+VAL_SIZE+TEST_SIZE

X1, X2, X3 = data1[:SPLIT_1], data1[SPLIT_1:SPLIT_2], data1[SPLIT_2:]
Y1, Y2, Y3 = label1[:SPLIT_1], label1[SPLIT_1:SPLIT_2], label1[SPLIT_2:]

print(f"X1.shape: {X1.shape}")
print(f"Y1.shape: {Y1.shape}")
print(f"X2.shape: {X2.shape}")
print(f"Y2.shape: {Y2.shape}")
print(f"X3.shape: {X3.shape}")
print(f"Y3.shape: {Y3.shape}")

X1.shape: (10781, 80, 80, 1)
Y1.shape: (10781, 5, 1)
X2.shape: (2068, 80, 80, 1)
Y2.shape: (2068, 5, 1)
X3.shape: (938, 80, 80, 1)
Y3.shape: (938, 5, 1)


In [20]:
# X1
# train_x, train_y, val_x, val_y, test_x, test_y = X1, Y1, X2, Y2, X3, Y3

# 2. TRAIN

### 2-1. MODEL

In [21]:
## ---------------------------------------------------------------- IN
input = Input(name="I", shape=(INPUT_SHAPE), batch_size=BATCH_SIZE)

## ---------------------------------------------------------------- 1
c1_1 = Conv2D(name="C2_1.1", filters=32, kernel_size=(3, 3), strides=2,
              padding='same', activation='relu')(input)
c1_2 = DepthwiseConv2D(name="DC2_1.2", kernel_size=32, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c1_1)
c1_3 = Conv2D(name="C2_1.3", filters=24, kernel_size=(1, 1), strides=1, padding='same')(c1_2)

## ---------------------------------------------------------------- 2
c2_1 = Conv2D(name="C2_2.1", filters=144, kernel_size=(1, 1), strides=1, padding='same', activation='relu')(c1_3)
c2_2 = DepthwiseConv2D(name="DC2_2.2", kernel_size=144, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c2_1)
c2_3 = Conv2D(name="C2_2.3", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c2_2)
## ---------------------------------------------------------------- 3
c3_1 = Conv2D(name="C2_3.1", filters=192, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c2_3)
c3_2 = DepthwiseConv2D(name="DC2_3.2", kernel_size=192, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c3_1)
c3_3 = Conv2D(name="C2_3.3", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c3_2)
c3_4 = Add(name="ADD_1")([c3_3, c2_3])

## ---------------------------------------------------------------- 4
c4_1 = Conv2D(name="C2_4.1", filters=192, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c3_4)
c4_2 = DepthwiseConv2D(name="DC2_4.2", kernel_size=192, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c4_1)
c4_3 = Conv2D(name="C2_4.3", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c4_2)
c4_4 = Add(name="ADD_2")([c4_3, c3_4])

## ---------------------------------------------------------------- 5
c5_1 = Conv2D(name="C2_5.1", filters=192, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c4_4)
c5_2 = DepthwiseConv2D(name="DC2_5.2", kernel_size=192, strides=2,
                       depth_multiplier=1, padding='same', activation='relu')(c5_1)
c5_3 = Conv2D(name="C2_5.3", filters=64, kernel_size=(1, 1), strides=1, padding='same')(c5_2)

## ---------------------------------------------------------------- 6
c6_1 = Conv2D(name="C2_6.1", filters=384, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c5_3)
c6_2 = DepthwiseConv2D(name="CD2_6.1", kernel_size=384, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c6_1)
c6_3 = Conv2D(name="C2_6.3", filters=64, kernel_size=(1, 1), strides=1, padding='same')(c6_2)
c6_4 = Add(name="ADD_3")([c6_3, c5_3])

## ---------------------------------------------------------------- 7
c7_1 = Conv2D(name="C2_7.1", filters=384, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c6_4)
c7_2 = DepthwiseConv2D(name="DC2_7.2", kernel_size=384, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c7_1)
c7_3 = Conv2D(name="DC2_7.3", filters=64, kernel_size=(1, 1), strides=1, padding='same')(c7_2)
c7_4 = Add(name="ADD_4")([c7_3, c6_4])

## ---------------------------------------------------------------- 8
c8_1 = Conv2D(name="C2_8.1", filters=384, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c7_4)
c8_2 = DepthwiseConv2D(name="DC2_8.2", kernel_size=384, strides=1,
                       depth_multiplier=1, padding='same', activation='relu')(c8_1)
c8_3 = Conv2D(name="C2_8.3", filters=96, kernel_size=(1, 1), strides=1, padding='same')(c8_2)
c8_4 = Conv2D(name="C2_8.4", filters=576, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c8_3)

## ---------------------------------------------------------------- 9
c9_1 = Conv2D(name="C2_9.1", filters=128, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c8_4)
c9_2 = Conv2D(name="C2_9.2", filters=256, kernel_size=(3, 3), strides=2,
              padding='same', activation='relu')(c9_1)
c9_3 = Conv2D(name="C2_9.3", filters=128, kernel_size=(1, 1), strides=1,
              padding='same', activation='relu')(c9_2)
c9_4 = Conv2D(name="C2_9.4", filters=256, kernel_size=(3, 3), strides=2,
              padding='same', activation='relu')(c9_3)

## ---------------------------------------------------------------- CONFIDENCE
l1 = Conv2D(name="C2_L1", filters=12, kernel_size=(1, 1), strides=1, padding='same')(c5_1)
l11 = Reshape(name="R_1", target_shape=(4800, 1, 4))(l1)

l2 = Conv2D(name="C2_L2", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c8_4)
l22 = Reshape(name="R_2", target_shape=(3200, 1, 4))(l2)

l3 = Conv2D(name="C2_L3", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c9_2)
l33 = Reshape(name="R_3", target_shape=(800, 1, 4))(l3)

l4 = Conv2D(name="C2_L4", filters=32, kernel_size=(1, 1), strides=1, padding='same')(c9_4)
l44 = Reshape(name="R_4", target_shape=(200, 1, 4))(l4)

y1 = Concatenate(name="CC_1", axis=1)([l11, l22, l33, l44])
a = Reshape(name="R_a", target_shape=(9000, 4))(y1)

## ---------------------------------------------------------------- LOCALIZATION
l5 = Conv2D(name="C2_L5", filters=6, kernel_size=(1, 1), strides=1, padding='same')(c5_1)
l55 = Reshape(name="R_5", target_shape=(4800, 2))(l5)

l6 = Conv2D(name="C2_L6", filters=16, kernel_size=(1, 1), strides=1, padding='same')(c8_4)
l66 = Reshape(name="R_6", target_shape=(3200, 2))(l6)

l7 = Conv2D(name="C2_L7", filters=16, kernel_size=(1, 1), strides=1, padding='same')(c9_2)
l77 = Reshape(name="R_7", target_shape=(800, 2))(l7)

l8 = Conv2D(name="C2_L8", filters=16, kernel_size=(1, 1), strides=1, padding='same')(c9_4)
l88 = Reshape(name="R_8", target_shape=(200, 2))(l8)

y2 = Concatenate(name="CC_2", axis=1)([l55, l66, l77, l88])
r = Activation(name="A1_1", activation='sigmoid')(y2)

## ---------------------------------------------------------------- OOU
output = Concatenate(name="O")([a, r])

## ---------------------------------------------------------------- FINAL
model_v1 = keras.Model(input, output)


2022-09-29 01:33:21.043432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 01:33:21.047503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 01:33:21.047775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 01:33:21.048261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [22]:
NUM_CLASSES = 2
# first 2 dimensions: equal to width of the output from the bottleneck expand ReLU at (4,1) and (5,3) respectively.
# dimensions after second: determined by the convolutions written inside SSD (conv1_2, conv2_2, conv3_3, conv4_2)
# numBoxes = [3, 3, 3, 3, 3, 3]
numBoxes = [3, 3, 3, 3]
# layerWidths = [28, 14, 7, 4, 2, 1]
layerWidths = [8, 4, 2, 1]
assert len(numBoxes) == len(layerWidths)  # numBoxes for each layer and each layer has a specific width
outputChannels = NUM_CLASSES+1+4  # classes + background + cx,cy,h,w
assert outputChannels-NUM_CLASSES == 5

# should be equal to 1st dimension in output layer of SSD model
BOXES = sum([a*a*b for a, b in zip(layerWidths, numBoxes)])
centres = np.zeros((BOXES, 2))
hw = np.zeros((BOXES, 2))
# boxes = np.zeros((BOXES, 4))
print(BOXES)

# train_x, train_y = convert(x_train, y_train)
# test_x, test_y = convert(x_test, y_test)
# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)

# checking if the inputs prepared are correct or not
r = np.random.randint(0, X1.shape[0])
img = X1[r, :, :, :].copy()
img_y = Y1[r]

im = np.array(Image.fromarray(img.astype(np.uint8)))
_, ax = plt.subplots(1)
ax.imshow(im)

# find all boxes where class label is not background
idx = np.argwhere(img_y[:, 0] != NUM_CLASSES)[:, 0]
print('Number of boxes with IoU > threshold (0.5):', idx.shape[0])
print('Green box: ground truth. Red box: default boxes with IoU < threshold (0.5)')

# calculating the ground truth bounding boxes
gt = np.zeros(4, dtype=np.uint16)
gt[:2] = (img_y[idx[0], 1:3]+centres[idx[0], :2])
gt[2:] = (img_y[idx[0], 3:]+hw[idx[0], :])

# for some reason, x and y are inverted
rect = patches.Rectangle((gt[1]-gt[3]/2, gt[0]-gt[2]/2), gt[3], gt[2], linewidth=5, edgecolor='g', facecolor='none')
ax.add_patch(rect)

# showing all the boxes with IoU > 0.5
for i in idx:
    rect = patches.Rectangle((centres[i,1]-hw[i,1]/2, centres[i,0]-hw[i,0]/2), hw[i,1], hw[i,0], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

plt.show()


3150


TypeError: Cannot handle this data type: (1, 1, 1), |u1

In [ ]:
# label is not required here in the standard implementation
def localisation_loss(x, y, label):
    diff = K.abs(x-y)  # * K.switch(label == 10, label*1.0/BOXES, label)
    result = K.switch(diff < 1, 0.5*diff**2, diff-0.5)  # smooth L1
    return K.mean(result)


def confidence_loss(y, label):
    unweighted_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(label, y)
    # class_weights = tf.constant([[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0/BOXES]]*BOXES])
    # weights = tf.reduce_sum(class_weights * y, axis = -1)
    # weighted_loss = unweighted_loss * weights
    return K.mean(unweighted_loss)


## gt = [66, 92, 28, 28]
## gt = [64, 171, 28, 28]
def Loss(gt, y):
    # shape of gt is n * BOXES * 5
    # shape of y is n * BOXES * output_channels
    loss = 0
    loss += localisation_loss(y[:, :, -4:], gt[:, :, -4:], gt[:, :, 0:1])
    loss += confidence_loss(y[:, :, :-4], tf.cast(gt[:, :, 0], tf.int32))
    return loss


In [ ]:
# model_v1 = v1_1(INPUT_SHAPE)
model_v1.compile(loss=Loss, optimizer='adam', metrics=['accuracy'])

model_v1.summary()


In [ ]:
history = model_v1.fit(X1, Y1, epochs=25, validation_data=(X2, Y2))
# history = model_v1.fit(train_dataset, epochs=25, validation_data=test_dataset)

In [ ]:
# model_v1.save("mv1.h5")

In [ ]:
# tf.compat.v1.disable_eager_execution()


In [ ]:
class Bottleneck(keras.Model):
    def __init__(self, expansion, stride, block_id, filters, alpha=1, ):
        super(Bottleneck, self).__init__(name="Bottleneck_"+block_id)
        self.stride = stride
        self.expansion = expansion
        self.alpha = alpha
        self.output_channels = self.alpha*filters
        self.out = None  # there was some problem with the eager execution

        prefix = 'Bottleneck_{}_'.format(block_id)
        self.prefix = prefix
        # expansion
        self.expand_BN = layers.BatchNormalization(name=prefix+'expand_BN')
        self.expand_ReLU = layers.ReLU(max_value=6, name=prefix+'expand_ReLU')

        #conv
        self.Conv = layers.DepthwiseConv2D(kernel_size=3, padding='same', strides=self.stride, use_bias=False,
                                           name=prefix+'conv')
        self.Conv_BN = layers.BatchNormalization(name=prefix+'conv_BN')
        self.Conv_ReLU = layers.ReLU(max_value=6, name=prefix+'conv_ReLU')

        #project
        self.project = layers.Conv2D(filters=self.output_channels, kernel_size=1, use_bias=False, name='contract')
        self.project_BN = layers.BatchNormalization(name=prefix+'contract_BN')

        # dimensions need to be the same for residual connection
        self.residual = layers.Add(name=prefix+'residual')

    def build(self, input_shape):
        self.d = input_shape[-1]
        self.expand = layers.Conv2D(filters=self.expansion*self.d, kernel_size=1, use_bias=False,
                                    name=self.prefix+'expand')

    def call(self, inputs):
        x = self.expand(inputs)
        x = self.expand_BN(x)
        x = self.expand_ReLU(x)
        self.out = x

        x = self.Conv(x)
        x = self.Conv_BN(x)
        x = self.Conv_ReLU(x)

        x = self.project(x)
        x = self.project_BN(x)

        if self.output_channels == self.d and self.stride == 1:
            x = self.residual([inputs, x])

        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))

        return keras.Model(inputs=[x], outputs=self.call(x))

In [ ]:
#using the architecture mentioned in the paper
class MobileNetv2(keras.Model):
    def __init__(self, k=11):
        super(MobileNetv2, self).__init__()
        self.conv_inp = layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), padding='valid', use_bias=False, name='conv')
        self.k = k

        self.pad = layers.ZeroPadding2D(padding=2, name='pad')
        self.BN = layers.BatchNormalization(name='BN')
        self.ReLU = layers.ReLU(max_value=6, name='ReLU')

        self.B1_1 = Bottleneck(expansion=1, filters=16, stride=1, block_id='B1_1')

        self.B2_1 = Bottleneck(expansion=6, filters=24, stride=2, block_id='B2_1')
        self.B2_2 = Bottleneck(expansion=6, filters=24, stride=1, block_id='B2_2')

        self.B3_1 = Bottleneck(expansion=6, filters=32, stride=2, block_id='B3_1')
        self.B3_2 = Bottleneck(expansion=6, filters=32, stride=1, block_id='B3_2')
        self.B3_3 = Bottleneck(expansion=6, filters=32, stride=1, block_id='B3_3')

        self.B4_1 = Bottleneck(expansion=6, filters=64, stride=2, block_id='B4_1')
        self.B4_2 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_2')
        self.B4_3 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_3')
        self.B4_4 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_4')

        self.B5_1 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_1')
        self.B5_2 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_2')
        self.B5_3 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_3')

        self.B6_1 = Bottleneck(expansion=6, filters=160, stride=2, block_id='B6_1')
        self.B6_2 = Bottleneck(expansion=6, filters=160, stride=1, block_id='B6_2')
        self.B6_3 = Bottleneck(expansion=6, filters=160, stride=1, block_id='B6_3')

        self.B7_1 = Bottleneck(expansion=6, filters=320, stride=1, block_id='B7_1')

        self.conv_out = layers.Conv2D(filters=1280, kernel_size=1, strides=(1, 1), use_bias=False, name='conv_out')
        self.avgpool = layers.AveragePooling2D(pool_size=(7, 7), name='avg_pool')

        self.conv_seg = layers.Conv2D(filters=self.k, kernel_size=1, strides=(1, 1), use_bias=False, name='conv_seg')

    def call(self, inputs):
        x = self.conv_inp(inputs)
        x = self.BN(x)
        x = self.ReLU(x)

        x = self.B1_1(x)
        x = self.B2_1(x)
        x = self.B2_2(x)

        x = self.B3_1(x)
        x = self.B3_2(x)
        x = self.B3_3(x)

        x = self.B4_1(x)
        x = self.B4_2(x)
        x = self.B4_3(x)
        x = self.B4_4(x)

        x = self.B5_1(x)
        x = self.B5_2(x)
        x = self.B5_3(x)

        x = self.B6_1(x)
        x = self.B6_2(x)
        x = self.B6_3(x)

        x = self.B7_1(x)

        x = self.conv_out(x)
        x = self.avgpool(x)
        c4 = self.conv_seg(x)

        return c4

    def model(self):
        x = keras.Input(shape=(224, 224, 1))

        return keras.Model(inputs=x, outputs=self.call(x))

In [ ]:
# MobileNetv2().model().summary()

In [ ]:
class SSD(keras.Model):
    def __init__(self, numBoxes=[4, 6, 6, 6, 4, 4], layerWidth=[28, 14, 7, 4, 2, 1], k=10+1+4):
        super(SSD, self).__init__()
        self.classes = k
        self.featureMaps = 6
        self.MobileNet = MobileNetv2(k=k)

        # mark bottleneck_6_1 onwards as non trainable
        # for layer in self.MobileNet.layers[-7:]:
        #     layer.trainable = False

        # For bottleneck_5_3, mark layers beyond conv as non runnable
        # layers in bottleneck_5_3: ['Bottleneck_B5_3_expand_BN', 'Bottleneck_B5_3_expand_ReLU', 'Bottleneck_B5_3_conv', 'Bottleneck_B5_3_conv_BN',
        # 'Bottleneck_B5_3_conv_ReLU', 'contract', 'Bottleneck_B5_3_contract_BN', 'Bottleneck_B5_3_residual', 'Bottleneck_B5_3_expand']
        # for layer in self.MobileNet.layers[-8].layers[2:-1]:
        #     layer.trainable = False

        self.numBoxes = numBoxes
        self.layerWidth = layerWidth
        self.features = [None for _ in range(self.featureMaps)]
        self.classifiers = [None for _ in range(self.featureMaps)]

        self.conv1_1 = layers.Conv2D(256, 1, name='SSD_conv_1_1')
        self.conv1_2 = layers.Conv2D(512, 3, strides=(2, 2), padding='same', name='SSD_conv_1_2')

        self.conv2_1 = layers.Conv2D(128, 1, name='SSD_conv_2_1')
        self.conv2_2 = layers.Conv2D(256, 3, strides=(2, 2), padding='same', name='SSD_conv_2_2')

        self.conv3_1 = layers.Conv2D(128, 1, name='SSD_conv_3_1')
        self.conv3_2 = layers.Conv2D(256, 3, strides=(1, 1), name='SSD_conv_3_2')

        self.conv4_1 = layers.Conv2D(128, 1, name='SSD_conv_4_1')
        self.conv4_2 = layers.Conv2D(256, 2, strides=(1, 1), name='SSD_conv_4_2')  # changed the kernel size to 2 since the output of the previous layer has width 3

        self.conv = []
        self.reshape = []
        for i in range(self.featureMaps):
            self.conv.append(layers.Conv2D(self.numBoxes[i]*self.classes, 3, padding='same', name='Classification_'+str(i)))
            self.reshape.append(layers.Reshape((self.layerWidth[i]*self.layerWidth[i]*self.numBoxes[i], self.classes), name='Reshape_classification_'+str(i)))

    def build(self, input_shape):
        self.MobileNet.build(input_shape)

    def call(self, inputs):
        x = inputs
        x = self.MobileNet(x)

        # get the convolved images at different resolutions
        self.features[0] = self.MobileNet.get_layer('Bottleneck_B4_1').out
        self.features[1] = self.MobileNet.get_layer('Bottleneck_B5_3').out
        self.features[2] = self.conv1_2(self.conv1_1(self.features[1]))
        self.features[3] = self.conv2_2(self.conv2_1(self.features[2]))
        self.features[4] = self.conv3_2(self.conv3_1(self.features[3]))
        self.features[5] = self.conv4_2(self.conv4_1(self.features[4]))

        for i in range(self.featureMaps):
            # for each feature map, create predictions according to the number of boxes for that layer and the number of output channels
            x = self.conv[i](self.features[i])
            x = self.reshape[i](x)
            self.classifiers[i] = x

        # concatenate all the classifiers
        x = layers.concatenate(self.classifiers, axis=-2, name='concatenate')
        return x

    def model(self):
        x = keras.Input(shape=(224, 224, 1))

        return keras.Model(inputs=x, outputs=self.call(x))

### PROCESS LABEL

In [ ]:
NUM_CLASSES = 2
# first 2 dimensions: equal to width of the output from the bottleneck expand ReLU at (4,1) and (5,3) respectively.
# dimensions after second: determined by the convolutions written inside SSD (conv1_2, conv2_2, conv3_3, conv4_2)
numBoxes = [3, 3, 3, 3, 3, 3]
layerWidths = [28, 14, 7, 4, 2, 1]
assert len(numBoxes) == len(layerWidths)  # numBoxes for each layer and each layer has a specific width
outputChannels = NUM_CLASSES + 1 + 4  # classes + background + cx,cy,h,w
assert outputChannels - NUM_CLASSES == 5

In [ ]:
# model = SSD(numBoxes=numBoxes, layerWidth=layerWidths, k=outputChannels)
# model.model().summary()

In [ ]:
# model.save('temp.h5')

In [ ]:
MinScale = .1  # Min and Max scale given as percentage
MaxScale = 1.5
scales = [MinScale+x/len(layerWidths)*(MaxScale-MinScale) for x in range(len(layerWidths))]
scales = scales[::-1]  # reversing order because layerWidths go from high to low (lower to higher resoltuion)

asp = [0.5, 1.0, 1.5]
asp1 = [x**0.5 for x in asp]
asp2 = [1/x for x in asp1]

IMG_SIZE = 80

# should be equal to 1st dimension in output layer of SSD model
BOXES = sum([a*a*b for a, b in zip(layerWidths, numBoxes)])
centres = np.zeros((BOXES, 2))
hw = np.zeros((BOXES, 2))
boxes = np.zeros((BOXES, 4))
print(BOXES)

# calculating default box centres, height, width
idx = 0
for gridSize, numBox, scale in zip(layerWidths, numBoxes, scales):
    step_size = IMG_SIZE*1.0/gridSize
    for i in range(gridSize):
        for j in range(gridSize):
            pos = idx+(i*gridSize+j)*numBox
            # centre is the same for all aspect ratios(=numBox)
            centres[pos: pos+numBox, :] = i*step_size+step_size/2, j*step_size+step_size/2
            # height and width vary according to the scale and aspect ratio
            # zip asepct ratios and then scale them by the scaling factor
            hw[pos: pos+numBox, :] = np.multiply(gridSize*scale, np.squeeze(np.dstack([asp1, asp2]), axis=0))[:numBox,:]
    idx += gridSize*gridSize*numBox

# (x,y) co-ordinates of top left and bottom right
# This actually is not used anywhere. centres[] and hw[] are a good enough substitute
boxes[:, 0] = centres[:, 0]-hw[:, 0]/2
boxes[:, 1] = centres[:, 1]-hw[:, 1]/2
boxes[:, 2] = centres[:, 0]+hw[:, 0]/2
boxes[:, 3] = centres[:, 1]+hw[:, 1]/2


# calculate IoU for a set of search boxes and default boxes
def IoU(box1, box2):
    box1 = box1.astype(np.float64)
    box2 = box2.astype(np.float64)
    # find left and right co-ordinates of edges. Min should be less than Max for non zero overlap
    xmin = np.maximum(box1[:, 0], box2[:, 0])
    xmax = np.minimum(box1[:, 2], box2[:, 2])
    ymin = np.maximum(box1[:, 1], box2[:, 1])
    ymax = np.minimum(box1[:, 3], box2[:, 3])

    intersection = np.abs(np.maximum(xmax-xmin, 0)*np.maximum(ymax-ymin, 0))
    boxArea1 = np.abs((box1[:, 2]-box1[:, 0])*(box1[:, 3]-box1[:, 1]))
    boxArea2 = np.abs((box2[:, 2]-box2[:, 0])*(box2[:, 3]-box2[:, 1]))
    unionArea = boxArea1+boxArea2-intersection
    assert (unionArea > 0).all()
    iou = intersection/unionArea

    return iou


# give index of box corresponding to IoUs > threshold (=0.5)
def bestIoU(searchBox):
    return np.argwhere(IoU(np.matlib.repmat(searchBox, BOXES, 1), boxes) > 0.5)


def convert(x, y):
    MNIST_SIZE = x.shape[-1]

    # input image location on canvas
    corner = np.random.randint(IMG_SIZE-MNIST_SIZE, size=(x.shape[0], 2))

    # create blank canvas for inputs
    input = np.zeros((x.shape[0], IMG_SIZE, IMG_SIZE, 3))
    # put MNIST on canvas
    for i in range(x.shape[0]):
        lx = int(corner[i, 0])
        ly = int(corner[i, 1])
        input[i, lx:lx+MNIST_SIZE, ly:ly+MNIST_SIZE, :] = np.repeat(np.expand_dims(np.array(x[i, :, :]), axis=-1), 3, axis=-1)

    # for each default box, there are 5 values: class number and delta cx,cy,h,w
    output = np.zeros((y.shape[0], BOXES, 1+4))
    output[:, :, 0] = NUM_CLASSES  # defaulting class labels for all boxes to background initially
    for i in range(x.shape[0]):
        bbox = np.zeros(4)
        bbox[:2] = corner[i]
        bbox[2:] = corner[i]+(MNIST_SIZE, MNIST_SIZE)
        # for all default boxes which have IoU > threshold, set the delta values and class number
        box_idx = bestIoU(bbox).astype(np.uint16)
        output[i, box_idx, 0] = y[i]
        output[i, box_idx, 1] = (bbox[0]+bbox[2])/2.0-centres[box_idx, 0]
        output[i, box_idx, 2] = (bbox[1]+bbox[3])/2.0-centres[box_idx, 1]
        output[i, box_idx, 3] = MNIST_SIZE-hw[box_idx, 0]
        output[i, box_idx, 4] = MNIST_SIZE-hw[box_idx, 1]

    return input, output


### 2-2. COMPILE

In [ ]:
train_x, train_y = convert(X1, Y1)
test_x, test_y = convert(X2, Y2)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:

# checking if the inputs prepared are correct or not
r = np.random.randint(0, X1.shape[0])
img = X1[r, :, :, :].copy()
img_y = Y1[r]

im = np.array(Image.fromarray(img.astype(np.uint8)))
fig, ax = plt.subplots(1)
ax.imshow(im)

# find all boxes where class label is not background
idx = np.argwhere(img_y[:, 0] != NUM_CLASSES)[:, 0]
print('Number of boxes with IoU > threshold (0.5):', idx.shape[0])
print('Green box: ground truth. Red box: default boxes with IoU < threshold (0.5)')

# calculating the ground truth bounding boxes
gt = np.zeros(4, dtype=np.uint16)
gt[:2] = (img_y[idx[0], 1:3]+centres[idx[0], :2])
gt[2:] = (img_y[idx[0], 3:]+hw[idx[0], :])

# for some reason, x and y are inverted
rect = patches.Rectangle((gt[1]-gt[3]/2, gt[0]-gt[2]/2), gt[3], gt[2], linewidth=5, edgecolor='g', facecolor='none')
ax.add_patch(rect)

# showing all the boxes with IoU > 0.5
for i in idx:
    rect = patches.Rectangle((centres[i][1]-hw[i, 1]/2, centres[i][0]-hw[i, 0]/2), hw[i, 1], hw[i, 0], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

plt.show()


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X1, Y1))
test_dataset = tf.data.Dataset.from_tensor_slices((X2, Y2))
print(train_dataset.element_spec)
print(test_dataset.element_spec)

SHUFFLE_BUFFER_SIZE = 60

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
# label is not required here in the standard implementation
def localisation_loss(x, y, label):
    # smooth L1
    diff = K.abs(x-y)  # * K.switch(label == 10, label*1.0/BOXES, label)
    result = K.switch(diff < 1, 0.5*diff**2, diff-0.5)
    return K.mean(result)


def confidence_loss(y, label):
    unweighted_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(label, y)
    # class_weights = tf.constant([[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0/BOXES]]*BOXES])
    # weights = tf.reduce_sum(class_weights * y, axis = -1)
    # weighted_loss = unweighted_loss * weights
    return K.mean(unweighted_loss)

## gt = [66, 92, 28, 28]
## gt = [64, 171, 28, 28]
def Loss(gt, y):
    # shape of gt is n * BOXES * 5
    # shape of y is n * BOXES * output_channels
    loss = 0
    loss += localisation_loss(y[:, :, -4:], gt[:, :, -4:], gt[:, :, 0:1])
    loss += confidence_loss(y[:, :, :-4], tf.cast(gt[:, :, 0], tf.int32))
    return loss


### 2-3. FIT

In [ ]:
base_learning_rate = 0.001
model_v1.compile(loss=Loss, optimizer=keras.optimizers.RMSprop(learning_rate=base_learning_rate), metrics=['accuracy'])

# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate), loss=Loss)


In [ ]:
# history = model_v1.fit(X1, Y1, epochs=25, validation_data=(X2, Y2))
history = model_v1.fit(train_dataset, epochs=25, validation_data=test_dataset)
# history = model.fit(X1, Y1, epochs=25, validation_data=(X2, Y2))


# 3. TEST

In [ ]:
model.evaluate(test_x, test_y)


In [ ]:
# create some sample data
# X, Y = convert(x_test, y_test)

# get prediction for one sample
y_pred = model.predict(X3)
y_pred.shape

OBJperCLASS = 10  # get top 10 results for each class


# get the confidence scores (with class values) and delta for the boxes. For each class, the top 10 values are used
def infer(Y):
    # classes are actually the index into the default boxes
    classes = np.zeros((OBJperCLASS, outputChannels-4), dtype=np.uint16)
    conf = np.zeros((OBJperCLASS, outputChannels-4))
    delta = np.zeros((OBJperCLASS, outputChannels-4, 4))
    class_predictions = softmax(Y[:, :outputChannels-4], axis=1)
    for i in range(outputChannels-4):
        classes[:, i] = bottleneck.argpartition(class_predictions[:, i], BOXES-1-10, axis=-1)[-OBJperCLASS:]
        conf[:, i] = class_predictions[classes[:, i], i]
        delta[:, i] = Y[classes[:, i], outputChannels-4:]
    return conf, classes, delta


# generate bounding boxes from the inferred outputs
def Bbox(confidence, box_idx, delta):
    # delta contains delta(cx,cy,h,w)
    bbox_centre = np.zeros((OBJperCLASS, outputChannels-4, 2))
    bbox_hw = np.zeros((OBJperCLASS, outputChannels-4, 2))
    for i in range(OBJperCLASS):
        bbox_centre[i, :, 0] = centres[box_idx[i]][:, 0]+delta[i, :, 0]
        bbox_centre[i, :, 1] = centres[box_idx[i]][:, 1]+delta[i, :, 1]
        bbox_hw[i, :, 0] = hw[box_idx[i]][:, 0]+delta[i, :, 2]
        bbox_hw[i, :, 1] = hw[box_idx[i]][:, 1]+delta[i, :, 3]
    return bbox_centre, bbox_hw


### 3-2. PLOT

In [ ]:
r = np.random.randint(TESTSIZE)

# top 10 predictions for each class
confidence, box_idx, delta = infer(y_pred[r])
bbox_centre, bbox_hw = Bbox(confidence, box_idx, delta)

im = np.array(Image.fromarray(X[r].astype(np.uint8)))
fig, ax = plt.subplots(1)
ax.imshow(im)

for i in range(outputChannels-4):
    # skipping backgrounds
    if i == NUM_CLASSES:
        continue
    color = 'r'
    # if a class is mentioned in the ground truth, color the boxes green
    if i in Y[r, :, 0]:
        color = 'g'
        print(i)

    # skip all the classes which have low confidence values
    if (confidence[:, i] > 0.5).any() or i in Y[r, :, 0]:
        for k in range(OBJperCLASS):
            print("{}: Confidence-{}\t\tCentre-{} Height,Width-{}".format(i, confidence[k, i], bbox_centre[k, i],
                                                                          bbox_hw[k, i]))

            # draw bounding box only if confidence scores are high
            if confidence[k, i] < 0.5:
                continue
            x = bbox_centre[k, i, 0]-bbox_hw[k, i, 0]/2
            y = bbox_centre[k, i, 1]-bbox_hw[k, i, 1]/2
            rect = patches.Rectangle((y, x), bbox_hw[k, i, 1], bbox_hw[k, i, 0], linewidth=1, edgecolor=color,
                                     facecolor='none')
            ax.add_patch(rect)

plt.show()
